In [1]:
!wget https://dphi-live.s3.eu-west-1.amazonaws.com/dataset/standford_cars.zip

In [2]:
!unzip standford_cars.zip

In [3]:
!cp -r ./standford_cars/train ./pictures

In [4]:
!cp ../input/carstraintest/predictions.csv ./predictions.csv

In [5]:
!cp ../input/carstraintest/train.csv ./train.csv

In [6]:
# !pip install -U pip

In [7]:
!pip install matplotlib==3.1.1

In [8]:
!pip install numpy --upgrade

In [9]:
!pip install ktrain --upgrade

In [10]:
!pip install scikit-learn --upgrade

In [11]:
import pandas as pd                                     # Data analysis and manipultion tool
import numpy as np                                      # Fundamental package for linear algebra and multidimensional arrays
import tensorflow as tf                                 # Deep Learning Tool
import os                                               # OS module in Python provides a way of using operating system dependent functionality
import cv2                                              # Library for image processing
from sklearn.model_selection import train_test_split    # For splitting the data into train and validation set

In [12]:
import ktrain
from ktrain import vision as vis

In [13]:
print(tf.__version__)

In [14]:
len(tf.config.list_physical_devices('GPU'))

In [15]:
# import tensorflow as tf 

if tf.test.gpu_device_name(): 

    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

else:

   print("Please install GPU version of TF")

In [16]:
print(ktrain.__version__)

In [17]:
vis.show_random_images('./pictures/')

In [18]:
vis.show_image('./pictures/Image_2.jpg')

In [19]:
vis.print_image_classifiers()

In [20]:
my_train = pd.read_csv('train.csv')

In [21]:
my_train

In [22]:
# vis.preprocess_csv('my_train.csv',)


# DATADIR = './pictures'
ORIGINAL_DATA = 'train.csv'
CONVERTED_DATA = 'trainCONVERTED.csv'
labels = vis.preprocess_csv(ORIGINAL_DATA, 
                           CONVERTED_DATA, 
                           x_col='filename', y_col='label')



In [23]:
converted_data = pd.read_csv('trainCONVERTED.csv')

In [24]:
converted_data.head()

In [25]:
labels

In [26]:
trn, val, preproc = vis.images_from_csv(
                          CONVERTED_DATA,
                         'filename',
                          directory='./pictures',
                          val_filepath = None,
                          label_columns = labels,
                          data_aug=vis.get_data_aug(horizontal_flip=True))

In [27]:
len(trn.filenames)

In [28]:
len(val.filenames)

In [29]:
model = vis.image_classifier('pretrained_resnet50', trn, val_data=val, 
                             #metrics=[tfa.metrics.F1Score(num_classes=N_CLASSES , average="micro")],
                             optimizer_name = 'adam',   #'sgd' ... 
                             freeze_layers=60,
                             multilabel=False,   # !!! (True, False or None)
                             pt_fc = [],
                             pt_ps = [],
                             verbose=1)

In [30]:
# model = vis.image_classifier('pretrained_resnet50', trn, val_data=val)
learner = ktrain.get_learner(model, train_data=trn, val_data=val, 
                             batch_size=64, workers=8)
# learner.freeze(2)

In [31]:
learner.lr_find(max_epochs=3)
learner.lr_plot()

In [32]:
learner.fit_onecycle(5 * 1e-4, 10)

In [33]:
learner.freeze(freeze_range=20)

In [34]:
learner.print_layers()

In [35]:
learner.fit_onecycle(1e-5, 1)

In [36]:
predictor = ktrain.get_predictor(learner.model, preproc)

In [37]:
vis.show_image('./pictures/Image_5694.jpg')

In [38]:
predictor.predict_filename('./pictures/Image_5694.jpg')

In [39]:
TEST_IMAGES = './pictures'

In [40]:
predictions = predictor.predict_folder(TEST_IMAGES, return_proba=False)

In [41]:
predictions[0]

In [42]:
len(predictions)

In [43]:
predictions[5692]

In [44]:
test_images = pd.read_csv('./predictions.csv')

In [45]:
def get_prediction(filename):
    filename = 'pictures/' + filename
    for prediction in predictions:
        if prediction[0] == filename:
            return prediction[1]
    return 'AAA'

In [46]:
get_prediction('Image_5627.jpg')

In [47]:
test_images = pd.read_csv('predictions.csv')
test_images

In [48]:
# test_images['filename'].apply(get_prediction)

In [49]:
test_images['label'] = test_images['filename'].apply(get_prediction)

In [50]:
test_images

In [51]:
test_images.to_csv("ktrain_predictions.csv", index = False)